In [84]:
from time import time

from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

tokenizer = AutoTokenizer.from_pretrained("sismetanin/rubert-toxic-pikabu-2ch")
model = AutoModelForSequenceClassification.from_pretrained("sismetanin/rubert-toxic-pikabu-2ch")


In [85]:
def is_toxic(text: str, threshold: float = 0.85):
    """
    Возвращает True/False — токсичное ли сообщение.
    threshold = порог токсичности (от 0 до 1).
    Необходимо подорбать порог
    """
    inputs = tokenizer(text, return_tensors="pt", truncation=True)
    with torch.no_grad():
        logits = model(**inputs).logits

    probs = torch.softmax(logits, dim=1)[0]
    toxic_prob = probs[1].item()

    return toxic_prob >= threshold, toxic_prob
start = time()
message = "я люблю нигеров"
flag, score = is_toxic(message)

print(flag, round(score,4))
print(time() - start)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


True 0.9888
0.05466127395629883


In [86]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_checkpoint = 'cointegrated/rubert-tiny-toxicity'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(29564, 312, padding_idx=0)
      (position_embeddings): Embedding(512, 312)
      (token_type_embeddings): Embedding(2, 312)
      (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-2): 3 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=312, out_features=312, bias=True)
              (key): Linear(in_features=312, out_features=312, bias=True)
              (value): Linear(in_features=312, out_features=312, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=312, out_features=312, bias=True)
              (LayerNorm): LayerNorm((312,), eps=1e-1

In [ ]:
def text2toxicity(text, aggregate=True):
    if isinstance(text, str):
        texts = [text]
        single_input = True
    else:
        texts = text
        single_input = False

    with torch.inference_mode():
        inputs = tokenizer(
            texts,
            return_tensors='pt',
            truncation=True,
            padding=True
        ).to(device)

        logits = model(**inputs).logits
        proba = torch.sigmoid(logits)  # shape: [batch_size, num_labels]

    if aggregate:
        # первый логит = "нет токсичности", последний = "тяжелая токсичность"
        # 1 - p(non_toxic) * (1 - p(hard_toxic))
        agg = 1.0 - proba[:, 0] * (1.0 - proba[:, -1])
        agg = agg.cpu().numpy()
        return agg[0] if single_input else agg

    proba = proba.cpu().numpy()
    return proba[0] if single_input else proba


In [102]:
start = time()
print(text2toxicity('я люблю нигеров', True))
print(time() - start)

0.93501186
0.01825428009033203
